In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import xgboost as xgb
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, train_test_split

In [ ]:
TRAIN_FEATHER = 'tmp/taxi-train-clean'
TEST_FEATHER = 'tmp/taxi-test-clean'
SUBM_CSV = 'data/nyc-taxi/sample_submission.csv'
OUTPUT_DIR = 'submissions/XGB/'
NSAMPLES = None
SEED = 1000

In [ ]:
train_df = pd.read_feather(TRAIN_FEATHER)
test_df = pd.read_feather(TEST_FEATHER)
subm_df = pd.read_csv(SUBM_CSV, index_col='key')

In [ ]:
train_df.shape, test_df.shape

# XGB

In [ ]:
#Cross-validation
xgb_params = {
    'max_depth': 8, 
    'eta':0.02
    'subsample': 1, 
    'colsample_bytree': 0.8,
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 1
}

In [ ]:
train = train_df #.sample(NSAMPLES, random_state=SEED)
y = train.fare_amount.copy()
train = train[test_df.columns]

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train, y, random_state=SEED, test_size=2_000_000)

In [ ]:
def XGBmodel(x_train,x_val,y_train,y_val,params):
    
    matrix_train = xgb.DMatrix(x_train,label=y_train)
    matrix_val = xgb.DMatrix(x_val,label=y_val)
    
    model=xgb.train(params=params,
                    dtrain=matrix_train,
                    verbose_eval=100,
                    num_boost_round=50_000, 
                    early_stopping_rounds=150,
                    evals=[(matrix_val,'val')])
    return model

model = XGBmodel(x_train,x_val,y_train,y_val,xgb_params)

In [ ]:
model.predict(test_df)

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
subm_df['fare_amount'] = y_preds
subm_df.to_csv(f'{OUTPUT_DIR}xgb.csv')
subm_df.head()